# KFold Cross-Validation
[A Gentle Introduction to k-fold Cross-Validation](https://machinelearningmastery.com/k-fold-cross-validation/)

Can use the sklearn api to implement quick cross validation splits based on index.


In [3]:
import numpy as np
from sklearn.model_selection import KFold


In [11]:
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
kfold = KFold(3, shuffle=True, random_state=1)
kfold = KFold(3, shuffle=False)

In [14]:
for train,test in kfold.split(data):
    print(f'train: {data[train]}, test: {data[test]}')

train: [0.3 0.4 0.5 0.6], test: [0.1  0.02]
train: [0.1  0.02 0.5  0.6 ], test: [0.3 0.4]
train: [0.1  0.02 0.3  0.4 ], test: [0.5 0.6]


## [Cross-Validation in PyTorch](https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/#implementing-k-fold-cross-validation-with-pytorch)




In [3]:
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

### Define a simple CNN


In [15]:
def reset_weights(m):
    '''Try resetting model weights to avoid weight leakage'''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Resent trainable parameters of layer = {layer}')
            layer.reset_parameters()

class SimpleConvNet(nn.Module):
    '''Simple Convolutional Neural Network'''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1,10,kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50,20),
            nn.ReLU(),
            nn.Linear(20,10)
        )

    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

In [8]:
k_folds = 5
num_epochs = 1
loss_function = nn.CrossEntropyLoss()

results = {}

torch.manual_seed(42)

In [10]:
# Prepare MNIST dataset by concatenating Train/Test part; we split later.
dataset_train_part = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor(), train=True)
dataset_test_part = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor(), train=False)
dataset = ConcatDataset([dataset_train_part, dataset_test_part])

3.0%

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

73.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Extracting /home/luc/MNIST/raw/train-images-idx3-ubyte.gz to /home/luc/MNIST/raw


102.8%
17.1%


Extracting /home/luc/MNIST/raw/train-labels-idx1-ubyte.gz to /home/luc/MNIST/raw



100.0%
112.7%


Extracting /home/luc/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/luc/MNIST/raw

Extracting /home/luc/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/luc/MNIST/raw



In [12]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Start print
print('--------------------------------')

--------------------------------


Everything in this next section is the same as normal except for the inclusion of a subset sampler that only draws from a subset of idxs from the entire dataset.

In [16]:
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    # The SubsetRandomSampler samples from the idxs without replacement
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network
    network = SimpleConvNet()
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = network(inputs)
        
        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
            
    # Process is complete.
    print('Training process has finished. Saving trained model.')

FOLD 0
--------------------------------
Starting epoch 1
Loss after mini-batch   500: 1.832
Loss after mini-batch  1000: 0.865
Loss after mini-batch  1500: 0.540
Loss after mini-batch  2000: 0.474
Loss after mini-batch  2500: 0.413
Loss after mini-batch  3000: 0.399
Loss after mini-batch  3500: 0.357
Loss after mini-batch  4000: 0.321
Loss after mini-batch  4500: 0.329
Loss after mini-batch  5000: 0.308
Loss after mini-batch  5500: 0.304
Training process has finished. Saving trained model.
FOLD 1
--------------------------------
Starting epoch 1
Loss after mini-batch   500: 1.841
Loss after mini-batch  1000: 0.802
Loss after mini-batch  1500: 0.513
Loss after mini-batch  2000: 0.425
Loss after mini-batch  2500: 0.391
Loss after mini-batch  3000: 0.359
Loss after mini-batch  3500: 0.316
Loss after mini-batch  4000: 0.344
Loss after mini-batch  4500: 0.336
Loss after mini-batch  5000: 0.308
Loss after mini-batch  5500: 0.319
Training process has finished. Saving trained model.
FOLD 2
---